In [18]:
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
%cd drive/MyDrive/tp_tpe_inf4248_groupe3/INF4248-Groupe3/

[Errno 2] No such file or directory: 'drive/MyDrive/tp_tpe_inf4248_groupe3/INF4248-Groupe3/'
/content/drive/MyDrive/tp_tpe_inf4248_groupe3/INF4248-Groupe3


# **Gradient Boosting classifier in python with Scikit-Learn**

 ### Membres du Groupe 3 
  Nom  | Prénom  | Matricule 
:------|:--------:|----------:
FORDJOU KAMGANG|LANDRY|19M2366
TEIDA NOUTSA| REMI RAOUL | 19M2447 
 NYA NJIKE| ARMEL | 21S2802   
 NJONOU KOUAYA| GABY | 19M2347   

## Implémentation d'un classifieur Gradient Boosting

> ## Classifieur Boosting régulier

>>## Description du jeu de données
Dans cette implémentation, nous utiliserons le jeu de données **Titanic**. Le but est de prédire pour chaque passager si oui ou non il a survécu au naufrage du titanic. C'est ainsi que nous considérerons l'attribut **"Survived"** comme la classe à prédire. La description du jeu de données est fournie ci-dessous:

Les données ont été divisées en deux groupes :

* jeu d'entrainement (train.csv)
* jeu de test (test.csv)

**Le jeu d'entrainement** doit être utilisé pour créer les modèles d'apprentissage automatique. Pour le jeu d'entrainement, nous fournissons le résultat (également connu sous le nom de **"vérité terrain"**) pour chaque passager. Le modèle sera basé sur des "caractéristiques" telles que le sexe et la classe des passagers. Vous pouvez également utiliser l'ingénierie des fonctionnalités pour créer de nouvelles fonctionnalités.

**Le jeu de test** doit être utilisé pour voir dans quelle mesure votre modèle fonctionne sur des données invisibles. Pour l'ensemble de test, nous ne fournissons pas la **"vérité terrain"** pour chaque passager. C'est votre travail de prédire ces résultats. Pour chaque passager de l'ensemble de test, utilisez le modèle que vous avez formé pour prédire s'il a survécu ou non au naufrage du Titanic.

Nous incluons également **gender_submission.csv**, un ensemble de prédictions qui supposent que toutes les passagères et seulement des femmes survivent, comme exemple de ce à quoi un fichier de soumission devrait ressembler.

**Dictionnaire de données**

  Variable  | Definition  | Key 
:------|:--------:|----------:
survival|Survival|0=No, 1=Yes
pclass|Ticket class|1=1st, 2=2nd, 3=3rd
sex|Sex|
Age|Age in years|
sibsp|# of siblings / spouses aboard the Titanic|
parch|# of parents / children aboard the Titanic|
ticket|Ticket number	|
fare|Passenger fare	|
cabin|Cabin number	|
embarked|Port of Embarkation	|C = Cherbourg, Q = Queenstown, S = Southampton

**Notes sur les varibles**

* **pclass**: A proxy for socio-economic status (SES)

>> 1st = Upper

>> 2nd = Middle

>> 3rd = Lower

* **age**: Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5

* **sibsp**: The dataset defines family relations in this way...

>> Sibling = brother, sister, stepbrother, stepsister

>> Spouse = husband, wife (mistresses and fiancés were ignored)

* **parch**: The dataset defines family relations in this way...

>> Parent = mother, father

>> Child = daughter, son, stepdaughter, stepson

>> Some children travelled only with a nanny, therefore parch=0 for them.

## Importation des bibliothèques

In [30]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier

## Chargement des données

In [31]:
train_data = pd.read_csv("titanic/train.csv", sep=',')
test_data = pd.read_csv("titanic/test.csv", sep=',')

In [32]:
# On visualise les 5 premières vaeurs de l'ensemble d' entrainement
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [33]:
# On visualise les 5 premières vaeurs de l'ensemble de test
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


## Prétraitement des données

In [34]:
y_train = train_data["Survived"]                            # Nous stockons la colonnes "Survived" dans un vecteur
train_data.drop(labels="Survived", axis=1, inplace=True)    # Nous supprimons la colonnes "Survived" sur le jeu d'entrainement

# Nous créons un jeu de données issu de la concaténation des jeux d'entrainement et de test
full_data = train_data.append(test_data)

# Nous supprimons les colonnes qui ne nous serviront pas
drop_columns = ["Name", "Age", "SibSp", "Ticket", "Cabin", "Parch", "Embarked"]
full_data.drop(labels=drop_columns, axis=1, inplace=True)

# Nous transformons les valeurs de la colonne "Sex" en entier tout en remplaçant les valeurs manquantes par des 0
full_data = pd.get_dummies(full_data, columns=["Sex"])
full_data.fillna(value=0.0, inplace=True)

# Nous séparons à nouveau le jeu de données en 2 paties: une pour l'entrainement et l'autre pour le test
X_train = full_data.values[0:891]
X_test = full_data.values[891:]

# Nous normalisons les données avec une normalisation MinMax
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



<ipython-input-34-795de39160b4>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_data = train_data.append(test_data)


In [79]:
"""
Nous séparons les données d'apprentissage en deux parties suivant les proportons: 70% pour l'entrainement et 30% pour la validation du modèle. 
"""

state = 12  # Ici nous définisons une graine  
test_size = 0.3  # Ici nous fixons la propoortion des données pour la validation à 30% 
  
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
                                                  test_size=test_size, random_state=state)

## Implémentation du classifieur Gradient Boosting

In [80]:
"""
Dans cette section, nous évaluons pour plusieurs taux de variations, la précision du modèle GradientBoostingClassifier sur le jeu d'apprentissage et le jeux de validation.
L'idée est de sélectionner le taux d'apprentissage qui fournit la plus meilleure précision
"""

lr_list = [0.05, 0.075, 0.1, 0.25, 0.5, 0.75, 1]    # Nous définissons une liste de taux d'apprentissage que nous testerons pour en déterminier le meilleur.

for learning_rate in lr_list:
    gb_clf = GradientBoostingClassifier(n_estimators=20, learning_rate=learning_rate, max_features=2, max_depth=2, random_state=0)
    gb_clf.fit(X_train, y_train)

    print("Learning rate: ", learning_rate)
    print("Accuracy score (training): {0:.3f}".format(gb_clf.score(X_train, y_train)))
    print("Accuracy score (validation): {0:.3f}".format(gb_clf.score(X_val, y_val)))
    print('')

Learning rate:  0.05
Accuracy score (training): 0.889
Accuracy score (validation): 0.778

Learning rate:  0.075
Accuracy score (training): 0.944
Accuracy score (validation): 0.667

Learning rate:  0.1
Accuracy score (training): 0.944
Accuracy score (validation): 0.667

Learning rate:  0.25
Accuracy score (training): 1.000
Accuracy score (validation): 0.778

Learning rate:  0.5
Accuracy score (training): 1.000
Accuracy score (validation): 0.778

Learning rate:  0.75
Accuracy score (training): 1.000
Accuracy score (validation): 0.778

Learning rate:  1
Accuracy score (training): 1.000
Accuracy score (validation): 0.889



In [81]:
"""
Reprenons la section ci-dessus pour observer le comportement de la précision du modèle
"""

lr_list = [0.05, 0.075, 0.1, 0.25, 0.5, 0.75, 1]    # Nous définissons une liste de taux d'apprentissage que nous testerons pour en déterminier le meilleur.

for learning_rate in lr_list:
    gb_clf = GradientBoostingClassifier(n_estimators=20, learning_rate=learning_rate, max_features=2, max_depth=2, random_state=0)
    gb_clf.fit(X_train, y_train)

    print("Learning rate: ", learning_rate)
    print("Accuracy score (training): {0:.3f}".format(gb_clf.score(X_train, y_train)))
    print("Accuracy score (validation): {0:.3f}".format(gb_clf.score(X_val, y_val)))
    print('')

Learning rate:  0.05
Accuracy score (training): 0.889
Accuracy score (validation): 0.778

Learning rate:  0.075
Accuracy score (training): 0.944
Accuracy score (validation): 0.667

Learning rate:  0.1
Accuracy score (training): 0.944
Accuracy score (validation): 0.667

Learning rate:  0.25
Accuracy score (training): 1.000
Accuracy score (validation): 0.778

Learning rate:  0.5
Accuracy score (training): 1.000
Accuracy score (validation): 0.778

Learning rate:  0.75
Accuracy score (training): 1.000
Accuracy score (validation): 0.778

Learning rate:  1
Accuracy score (training): 1.000
Accuracy score (validation): 0.889



**Observations :** 
* le taux d'apprentissage 0.5 produit la meilleure précision sur le jeu de validation
* La valeur de la précision varie entre deux exécutions avec les mêmes paramètres
* La précision atteint la valeur 1 sur le jeu d'apprentissage




In [37]:
"""
Dans cette section, nous implémentons le GradientBoostinfClassifier avec le taux d'apprentissage 0.5 qui s'est avéré être le meilleur de la 
liste que nous avons définie.
"""

gb_clf2 = GradientBoostingClassifier(n_estimators=20, learning_rate=0.5, max_features=2, max_depth=2, random_state=0)
gb_clf2.fit(X_train, y_train)
predictions = gb_clf2.predict(X_val)

print("Confusion Matrix:")
print(confusion_matrix(y_val, predictions))

print("Classification Report")
print(classification_report(y_val, predictions))

Confusion Matrix:
[[142  19]
 [ 42  65]]
Classification Report
              precision    recall  f1-score   support

           0       0.77      0.88      0.82       161
           1       0.77      0.61      0.68       107

    accuracy                           0.77       268
   macro avg       0.77      0.74      0.75       268
weighted avg       0.77      0.77      0.77       268



## Implémentaion du Classifieur XGBoost

In [46]:
from xgboost import XGBClassifier   # Importation de la classe XGBClassifier
xgb_clf = XGBClassifier()           # Instanciation d'un objet XGBClassifier
xgb_clf.fit(X_train, y_train)       # Entrainement du modèle sur le jeu d'entrainement dont les observations sont séparées des classes

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [48]:
"""
Evaluation du modèle
"""
score = xgb_clf.accuracy_score(X_val, y_val)
print("Score: {0:.5f}".format(score))

Score: 0.74627


## Observations
* Face à un jeu de données, il est très intéressant de tester plusieurs taux d'apprentissages pour choisir le plus adapté au problème que l'on souhaite résoudre.
* La précision atteint la valeur 1 sur le jeu d'apprentissage. Ce caractérise le surapprentissage

## Points bien compris
* **Gradient boosting** est une implémentation de l'algorithme **Boosting** basée sur la minimisation d'une fonction de perte par la méthode du gradient
* Elle peut aussi bien être appliquée à la classification qu' à la regression
* Pour les problèmes de classification, la loss-function la plus utilisée est l'entropie croisée et pour le problèmes de regression, c'est la moyenne des carrés des erreurs
* De manière générale, Gradient Boosting consiste à initialiser des classifieurs de faible performance (weak learners) et en construire de nouveaux de manière itérative ayant à chaque fois une meilleure performance que les précédents. Ces nouveaux classifieurs sont formés sur les données mal classées et plus données sont mal classées, plus le poids du classifieurs qui se forme sur elles est élevé. Au fur et à mesure de l'apprentissage, les poids des modèles qui classent bien sont gelés dans le but d'être conservés. L'apprentissage consiste donc à construire des classifieurs en minimisant l'erreur en utilisant la descente du gradient.
* Les weak learners utilisés dans Gradient Boosting sont des **arbres de décision binaires**
* Graident Boosting est très succeptible de surapprendre. Ceci est dû au fait qu'il utilise des arbres de décision binaires qui eux-mêmes sont très succeptibles de surapprendre. Mais heureseument, il existe de nombres méthodes pour y remédier. On peut entre autre citer: **penalized learning**, **tree contraints**, **Random sampling / stochastic Boosting**, **Shrinkage / Weighted Update** et **XGBoost**
* **XGBoost (eXtrem Gradient Boosting)** est une adaptation très rapide de Gradient Boosting qui produit de très bons résultats en très peu de temps et consomme moins d'espace mémoire.

## Points mal compris / questions
* Nous aierions comprendre d'où XGBoost tire sa grande vitesse couplée à sa très bonne performance. Autrement dit: Comment est-ce qu'il fait pour produire de très bon résultats en très peu de temps et en consommant très peu de mémoire ?
* Dans le tutoriel,  il est mentionné que les algorithe de Gradient Boosting sont très efficaces sur des jeux de données complexes. Nous aimerions mieux comprendre la notion de **jeu de données complexe**. Comment les reconnaît-on ?